## 한빛아카데미 - 새로나온 책
- https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1 (메인)
- https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B3261280549 (서브)
- a href='/academy/books/book_view.html?p_code=B3261280549' (알 수 있는 정보)
- 제목, 저자, 출간일, 쪽수, 가격, 역자

In [12]:
import requests
import pandas as pd 
from tqdm import tqdm
from bs4 import BeautifulSoup

- Step 1

In [13]:
base_url = 'https://www.hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

- Step 2

In [14]:
lis = soup.select('.sub_book_list')
len(lis)

20

- Step 3

In [15]:
li = lis[0]
sub_href = li.select_one('.book_tit > a')['href']
sub_href

'/academy/books/book_view.html?p_code=B3261280549'

- sub page에서 정보 추출

In [16]:
sub_url = base_url + sub_href
sub_res = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_res.text, 'html.parser')

In [17]:
title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
title

'관리회계 7판'

In [18]:
# 책 정보의 딕셔너리 생성
sub_lis = sub_soup.select('.info_list > li')
info_dict = {}
for sub_li in sub_lis[:-1]:
    key = sub_li.find('strong').get_text().strip()
    value = sub_li.find('span').get_text().strip()
    info_dict[key[:-2]] = value
info_dict

{'저자': 'James Jiambalvo',
 '번역': '손성진',
 '출간': '2023-09-01',
 '페이지': '572 쪽',
 'ISBN': '9791156646761',
 '물류코드': '4676'}

In [19]:
price = int(sub_soup.select_one('.pbr > strong').string.replace(',',''))
price

36000

In [20]:
'a' in info_dict.keys()

False

- Step 4.

In [21]:
data = []
for page in tqdm(range(1, 24)):
    url = f'{base_url}{main_url}?page={page}'
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    lis = soup.select('.sub_book_list')
    for li in lis:
        sub_href = li.select_one('.book_tit > a')['href']
        sub_url = base_url + sub_href
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        
        title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        sub_lis = sub_soup.select('.info_list > li')
        info_dict = {}
        for sub_li in sub_lis[:-1]:
            key = sub_li.find('strong').get_text().strip()
            value = sub_li.find('span').get_text().strip()
            info_dict[key[:-2]] = value
        price = int(sub_soup.select_one('.pbr > strong').string.replace(',',''))
        
        author = info_dict['저자']
        translator = info_dict['번역'] if '번역' in info_dict.keys() else ' '
        date = info_dict['출간']
        page = int(info_dict['페이지'][:-2])

        data.append({'제목':title, '저자':author, '역자':translator, '출간일':date,
                    '페이지수':page, '가격':price})

100%|██████████| 23/23 [02:02<00:00,  5.31s/it]


In [22]:
df = pd.DataFrame(data)
df.tail()

,제목,저자,역자,출간일,페이지수,가격
448,문제해결력을 키우는 이공계 글쓰기 : 발표와 토론까지,한성우,,2013-02-28,396,18000
449,Essential Calculus,James Stewart,,2012-07-08,960,36000
450,의사결정회계,정재권,,2011-08-25,506,28000
451,"IT CookBook, 디지털 통신 시스템 : Digital Communicatio...","정호영 , 임승각",,2011-01-31,576,25000
452,인간관계론,"유기현 , 송병선 , 권용만",,2008-09-15,415,25000
